In [27]:
#! pip install fairlearn
#! pip install lightgbm
#! pip install optuna

In [28]:
from sklearn.metrics import f1_score, confusion_matrix, make_scorer, accuracy_score, recall_score
from sklearn.model_selection import cross_val_score, train_test_split, cross_validate
from fairlearn.datasets import fetch_adult
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

import numpy as np
import optuna
import tqdm as notebook_tqdm
from metrics import (
    equality_opportunity_difference,
    predictive_equality_difference,
    metrics
)
from fairlearn.metrics import demographic_parity_difference


In [29]:
data = fetch_adult(as_frame=True)
X_raw = data.data
y = (data.target == ">50K") * 1
A = X_raw["sex"]

numeric_transformer = Pipeline(
    steps=[
        ("impute", SimpleImputer()),
        ("scaler", StandardScaler()),
    ]
)
categorical_transformer = Pipeline(
    [
        ("impute", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore")),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, selector(dtype_exclude="category")),
        ("cat", categorical_transformer, selector(dtype_include="category")),
    ]
)


/home/avasquez/miniconda3/envs/fairenv/lib/python3.10/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [30]:
results = []


In [34]:

directions = ['minimize', 'maximize']
metric_scorer_decorated =  metrics(recall_score, predictive_equality_difference, sensitive_col = 'sex')
for sim in [0,1,2,3]:
    print(sim)
    def objective(trial):

        (X_train, X_test, y_train, y_test, A_train, A_test) = train_test_split(
        X_raw, y, A, test_size=0.8, random_state=sim, stratify=y
        )

        X_train = X_train.reset_index(drop=True)
        X_test = X_test.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)
        y_test = y_test.reset_index(drop=True)
        A_train = A_train.reset_index(drop=True)
        A_test = A_test.reset_index(drop=True)

        classifier_name = trial.suggest_categorical("classifier", ["logit", "RF", 'GBM','LGBM'])

        if classifier_name == "logit":        
            params = {
                "penalty" : trial.suggest_categorical('logit_penalty', ['l1','l2']),
                "C" : trial.suggest_float('logit_c', 0.001, 10),
                #"solver" : trial.suggest_categorical('logit_solver', ['saga','lbfgs']),
                "max_iter": 2000,
                "solver" : 'saga'
                }
            classifier = LogisticRegression(**params)
        
        elif classifier_name =="RF":
            params = {
                'n_estimators': trial.suggest_int("rf_n_estimators", 100, 1000),
                'criterion': trial.suggest_categorical("rf_criterion", ['gini', 'entropy']),
                'max_depth': trial.suggest_int("rf_max_depth", 1, 4),
                'min_samples_split': trial.suggest_float("rf_min_samples_split", 0.01, 1),
                }
            classifier = RandomForestClassifier(**params)

        elif classifier_name =="LGBM":
            params = {
                'n_estimators': trial.suggest_int("lgbm_n_estimators", 20, 10000),
                'num_leaves': trial.suggest_int("lgbm_num_leaves", 10, 1000),
                'max_depth': trial.suggest_int("lgbm_max_depth", 2, 20),
                'min_child_samples': trial.suggest_int("lgbm_min_child_samples", 5, 300),
                'learning_rate': trial.suggest_float('lgbm_learning_rate', 1e-5, 1e-2),
                'boosting_type': trial.suggest_categorical("lgbm_boosting_type", ['goss', 'gbdt'])
                }
            classifier = LGBMClassifier(**params)  
        
        elif classifier_name =="GBM":
            params = {
                'n_estimators': trial.suggest_int("gbm_n_estimators", 100, 1000), 
                'criterion': trial.suggest_categorical("gbm_criterion", ['squared_error', 'friedman_mse']),
                'max_depth': trial.suggest_int("gbm_max_depth", 1, 4),
                'min_samples_split': trial.suggest_int("gbm_min_samples_split", 5, 300),
                }
            classifier = GradientBoostingClassifier(**params)            
        
        else:
            None
        

        pipeline = Pipeline(
            steps=[
                ("preprocessor", preprocessor),
                ("classifier", classifier),
            ]
        )
                
        scores = cross_validate(
                pipeline, 
                X_train,
                y_train, 
                cv=5,
                scoring = metric_scorer_decorated,
                return_train_score=True)

        fair_metric = scores['test_fairness'].mean()
        model_metric = scores['test_model'].mean()

        return fair_metric, model_metric
    
    
    
    study = optuna.create_study(
        directions = directions, 
        pruner = optuna.pruners.SuccessiveHalvingPruner(), 
        sampler = optuna.samplers.TPESampler() 
        )
    study.optimize(objective, n_trials=100)
    print("Number of finished trials: ", len(study.trials))
    results.append(study)

[I 2023-05-15 16:44:38,736] A new study created in memory with name: no-name-7ce451e6-71d9-4884-9724-351981b1bade


0


/home/avasquez/miniconda3/envs/fairenv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/avasquez/miniconda3/envs/fairenv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2023-05-15 16:45:05,832] Trial 0 finished with values: [0.07460720729657666, 0.5956349859990117] and parameters: {'classifier': 'logit', 'logit_penalty': 'l1', 'logit_c': 3.098048049235774}. 
[I 2023-05-15 16:45:13,505] Trial 1 finished with values: [0.07426764742713346, 0.5964860264646132] and parameters: {'classifier': 'logit', 'logit_penalty': 'l2', 'logit_c': 7.171695954929824}. 
[I 2023-05-15 16:45:36,192] Trial 2 finished with values: [0.08775513683548257, 0.5768178407364705] and parameters: {'classifier': 'LGBM', 'lgbm_n_estimators': 7556, 'lgbm_num_leaves': 519, 'lgbm_max_dept

Number of finished trials:  100
1


[I 2023-05-15 17:13:10,789] Trial 0 finished with values: [0.07954047534368182, 0.6427030143304233] and parameters: {'classifier': 'GBM', 'gbm_n_estimators': 875, 'gbm_criterion': 'squared_error', 'gbm_max_depth': 3, 'gbm_min_samples_split': 100}. 
[I 2023-05-15 17:13:38,567] Trial 1 finished with values: [0.0865532589907491, 0.5879326122366807] and parameters: {'classifier': 'logit', 'logit_penalty': 'l1', 'logit_c': 5.455591436384834}. 
[I 2023-05-15 17:14:16,657] Trial 2 finished with values: [0.09655715791325234, 0.6037702007723421] and parameters: {'classifier': 'LGBM', 'lgbm_n_estimators': 9536, 'lgbm_num_leaves': 654, 'lgbm_max_depth': 17, 'lgbm_min_child_samples': 181, 'lgbm_learning_rate': 0.0023253592964109695, 'lgbm_boosting_type': 'goss'}. 
[I 2023-05-15 17:14:24,369] Trial 3 finished with values: [0.08699142765689025, 0.588360877761306] and parameters: {'classifier': 'logit', 'logit_penalty': 'l2', 'logit_c': 9.52797135710152}. 
[I 2023-05-15 17:16:23,587] Trial 4 finished

Number of finished trials:  100
2


[I 2023-05-15 18:08:27,337] Trial 0 finished with values: [0.0, 0.0] and parameters: {'classifier': 'RF', 'rf_n_estimators': 791, 'rf_criterion': 'gini', 'rf_max_depth': 1, 'rf_min_samples_split': 0.3449585866967549}. 
[I 2023-05-15 18:08:29,093] Trial 1 finished with values: [0.0, 0.0] and parameters: {'classifier': 'RF', 'rf_n_estimators': 133, 'rf_criterion': 'gini', 'rf_max_depth': 3, 'rf_min_samples_split': 0.8930532406514512}. 
[I 2023-05-15 18:08:37,860] Trial 2 finished with values: [0.0875212449736727, 0.6088929153168982] and parameters: {'classifier': 'logit', 'logit_penalty': 'l2', 'logit_c': 7.430067981698132}. 
/home/avasquez/miniconda3/envs/fairenv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/avasquez/miniconda3/envs/fairenv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the co

Number of finished trials:  100
3


[I 2023-05-15 18:30:50,230] Trial 0 finished with values: [0.0002127659574468085, 0.07318581965262907] and parameters: {'classifier': 'LGBM', 'lgbm_n_estimators': 6169, 'lgbm_num_leaves': 337, 'lgbm_max_depth': 5, 'lgbm_min_child_samples': 221, 'lgbm_learning_rate': 7.864797248917248e-05, 'lgbm_boosting_type': 'goss'}. 
[I 2023-05-15 18:31:28,283] Trial 1 finished with values: [0.06812245904325334, 0.642722231373195] and parameters: {'classifier': 'GBM', 'gbm_n_estimators': 916, 'gbm_criterion': 'squared_error', 'gbm_max_depth': 3, 'gbm_min_samples_split': 30}. 
[I 2023-05-15 18:31:47,793] Trial 2 finished with values: [0.08471499467321077, 0.6050632332216914] and parameters: {'classifier': 'logit', 'logit_penalty': 'l1', 'logit_c': 2.92194758992499}. 
[I 2023-05-15 18:31:56,407] Trial 3 finished with values: [0.000648543869650725, 0.1459168359596625] and parameters: {'classifier': 'RF', 'rf_n_estimators': 619, 'rf_criterion': 'gini', 'rf_max_depth': 4, 'rf_min_samples_split': 0.225205

Number of finished trials:  100


In [35]:
import dill
file_name = 'recall-fpr-models-motpe-succesivehalving-100trials-4sim.pkl'
with open(file_name, 'wb') as file:
    dill.dump(results, file)
    print(f'Object successfully saved to "{file_name}"')

Object successfully saved to "recall-fpr-models-motpe-succesivehalving-100trials-4sim.pkl"
